In [1]:
import pandas as pd
import numpy as np
from numpy import mean
import random
import re
from tqdm.notebook import tqdm
from datetime import datetime
import string
from string import punctuation

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import os
import pyspark
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import functions 
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, length, split, lit, when, concat, isnull, mode, isnan, count, lower, upper, sum, first, regexp_replace, concat_ws, year, hour, minute, month, to_date, to_timestamp
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pymongo import MongoClient

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import googletrans
from googletrans import *
from langdetect import detect

from operator import add
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import warnings
warnings.filterwarnings('ignore')

2024-05-20 02:25:38.817881: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 02:25:39.130683: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 02:25:40.541824: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 02:25:44.162805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Read from hadoop with Pyspark

In [2]:
df = spark.read.csv('/CA2/ProjectTweets.csv', header=False).toDF("Indx","ID", "DATE", "FLAG", "USER", "TWEET")

## Write to MongoDB

In [3]:
#df = spark.read.csv('/CA2/ProjectTweets.csv', header=False).toDF("Indx","ID", "DATE", "FLAG", "USER", "TWEET")
# Start Spark Session
# spark = SparkSession.builder \
#     .appName("CSV to MongoDB") \
#     .getOrCreate()

# Define MongoDB connection
mongo_uri = "mongodb://localhost:27017/"
database_name = "Tweets"
collection_name = "tweets_collection"

# Convert spark df in a MongoDB readable format
data = df.toPandas().to_dict(orient='records')

# Connection to MongoDB
client = MongoClient(mongo_uri)
db = client[database_name]
collection = db[collection_name]

# Insert to MongoDB
collection.insert_many(data)
print('\n')
print('Data inserted')

# Close connection to MongoDB
client.close()

# Stop Spark Session
#spark.stop()



Data inserted


## Read from MongoDB

In [4]:
# # Start Spark Session
# spark = SparkSession.builder \
#     .appName("MongoDB to PySpark") \
#     .config("spark.mongodb.input.uri", "mongodb://localhost:27017/Tweets.tweets_collection") \
#     .getOrCreate()

mongo_uri = "mongodb://localhost:27017/Tweets.tweets_collection"

# Read from MongoDB
dfM = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_uri).load()
#df1 = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

# Check df
dfM.show(5)

# Check number of rows
print(dfM.count())

# Stop Spark Session
#spark.stop()

+--------------------+--------+----------+----+--------------------+---------------+--------------------+
|                DATE|    FLAG|        ID|Indx|               TWEET|           USER|                 _id|
+--------------------+--------+----------+----+--------------------+---------------+--------------------+
|Mon Apr 06 22:19:...|NO_QUERY|1467810369|   0|@switchfoot http:...|_TheSpecialOne_|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467810672|   1|is upset that he ...|  scotthamilton|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467810917|   2|@Kenichan I dived...|       mattycus|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467811184|   3|my whole body fee...|        ElleCTF|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467811193|   4|@nationwideclass ...|         Karoli|{664aa714d56681f7...|
+--------------------+--------+----------+----+--------------------+---------------+--------------------+
only showing top 5 rows



1600000


In [5]:
df1 = dfM.alias('df1')

In [6]:
df1.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- FLAG: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Indx: string (nullable = true)
 |-- TWEET: string (nullable = true)
 |-- USER: string (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)



## Data cleaning using Spark

In [7]:
# column_order = ["Indx", "ID", "DATE", "FLAG", "USER", "TWEET"]

# df1 = df1.select(column_order)

# df1 = df1.dropDuplicates().withColumn("Indx", df1["Indx"].cast("int")).orderBy(col("Indx"))

df1.show(5)
df1.printSchema()

+--------------------+--------+----------+----+--------------------+---------------+--------------------+
|                DATE|    FLAG|        ID|Indx|               TWEET|           USER|                 _id|
+--------------------+--------+----------+----+--------------------+---------------+--------------------+
|Mon Apr 06 22:19:...|NO_QUERY|1467810369|   0|@switchfoot http:...|_TheSpecialOne_|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467810672|   1|is upset that he ...|  scotthamilton|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467810917|   2|@Kenichan I dived...|       mattycus|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467811184|   3|my whole body fee...|        ElleCTF|{664aa714d56681f7...|
|Mon Apr 06 22:19:...|NO_QUERY|1467811193|   4|@nationwideclass ...|         Karoli|{664aa714d56681f7...|
+--------------------+--------+----------+----+--------------------+---------------+--------------------+
only showing top 5 rows

root
 |-- DATE: strin

In [8]:
column_order = ["Indx", "ID", "DATE", "FLAG", "USER", "TWEET"]

df1 = df1.select(column_order)

df1 = df1.dropDuplicates().withColumn("Indx", df1["Indx"].cast("int")).orderBy(col("Indx"))

# Split Date column
df1 = df1.withColumn("Day_of_week", split(df1["DATE"], " ")[0]) \
         .withColumn("Month", split(df1["DATE"], " ")[1]) \
         .withColumn("Day", split(df1["DATE"], " ")[2]) \
         .withColumn("Time", split(df1["DATE"], " ")[3]) \
         .withColumn("Timezone", split(df1["DATE"], " ")[4]) \
         .withColumn("Year", split(df1["DATE"], " ")[5])

# Map Months
m_map = {'Jan': '01', 'Feb': '02', 'Mar': '03',
         'Apr': '04', 'May': '05', 'Jun': '06',
         'Jul': '07', 'Aug': '08', 'Sep': '09',
         'Oct': '10', 'Nov': '11', 'Dec': '12'}

df1 = df1.withColumn("MONTH", 
                     when(df1["MONTH"] == "Jan", "01")
                     .when(df1["MONTH"] == "Feb", "02")
                     .when(df1["MONTH"] == "Mar", "03")
                     .when(df1["MONTH"] == "Apr", "04")
                     .when(df1["MONTH"] == "May", "05")
                     .when(df1["MONTH"] == "Jun", "06")
                     .when(df1["MONTH"] == "Jul", "07")
                     .when(df1["MONTH"] == "Aug", "08")
                     .when(df1["MONTH"] == "Sep", "09")
                     .when(df1["MONTH"] == "Oct", "10")
                     .when(df1["MONTH"] == "Nov", "11")
                     .when(df1["MONTH"] == "Dec", "12")
                     .otherwise(df1["MONTH"]))

# Maap weekdays
d_map = {'Mon': '01', 'Tue': '02', 'Wed': '03',
         'Thu': '04', 'Fri': '05', 'Sat': '06',
         'Sun': '07'}

df1 = df1.withColumn("DAY_OF_WEEK", 
                     when(df1["DAY_OF_WEEK"] == "Mon", "01")
                     .when(df1["DAY_OF_WEEK"] == "Tue", "02")
                     .when(df1["DAY_OF_WEEK"] == "Wed", "03")
                     .when(df1["DAY_OF_WEEK"] == "Thu", "04")
                     .when(df1["DAY_OF_WEEK"] == "Fri", "05")
                     .when(df1["DAY_OF_WEEK"] == "Sat", "06")
                     .when(df1["DAY_OF_WEEK"] == "Sun", "07")
                     .otherwise(df1["DAY_OF_WEEK"]))

# Create full DATE column
df1 = df1.withColumn("MONTH", df1["MONTH"].cast("string"))
df1 = df1.withColumn("Day", df1["Day"].cast("string"))
df1 = df1.withColumn("Year", df1["Year"].cast("string"))
df1 = df1.withColumn("Time", df1["Time"].cast("string"))
df1 = df1.withColumn("DATE", concat("Year", lit("-"), "MONTH", lit("-"), "Day", lit(" "), "Time"))
df1 = df1.withColumn("DATE", to_timestamp(df1["DATE"], "yyyy-MM-dd HH:mm:ss"))

# Drop columns
df1 = df1.select("Indx", "ID", "DATE", "DAY_OF_WEEK", "TWEET")

# Clean tweets
df1 = df1.withColumn("TWEET", lower(col("TWEET")))
df1 = df1.withColumn("TWEET", F.regexp_replace(col("TWEET"), r'@\w+', ''))
df1 = df1.withColumn("TWEET", F.regexp_replace(col("TWEET"), r'http\S+', ''))
df1 = df1.withColumn("TWEET", 
                     regexp_replace(col("TWEET"), r'[^\w\s]|quot|amp|\bwww\.\S+|\d+', ''))

# Count tweets with more than 140 characters
df1 = df1.withColumn("tweet_length", length("TWEET"))
print('Tweets with more than 140 characters:', df1.filter(length("TWEET") > 140).count())
df1.show(5)

Tweets with more than 140 characters: 2


+----+----------+-------------------+-----------+--------------------+------------+
|Indx|        ID|               DATE|DAY_OF_WEEK|               TWEET|tweet_length|
+----+----------+-------------------+-----------+--------------------+------------+
|   0|1467810369|2009-04-06 22:19:45|         01|   awww thats a b...|          74|
|   1|1467810672|2009-04-06 22:19:49|         01|is upset that he ...|         105|
|   2|1467810917|2009-04-06 22:19:53|         01| i dived many tim...|          76|
|   3|1467811184|2009-04-06 22:19:57|         01|my whole body fee...|          47|
|   4|1467811193|2009-04-06 22:19:57|         01| no its not behav...|          87|
+----+----------+-------------------+-----------+--------------------+------------+
only showing top 5 rows



In [9]:
df1.printSchema()

root
 |-- Indx: integer (nullable = true)
 |-- ID: string (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- TWEET: string (nullable = true)
 |-- tweet_length: integer (nullable = true)



In [10]:
# Clean tweets
df1 = df1.withColumn("TWEET", lower(col("TWEET")))
df1 = df1.withColumn("TWEET", F.regexp_replace(col("TWEET"), r'@\w+', ''))
df1 = df1.withColumn("TWEET", F.regexp_replace(col("TWEET"), r'http\S+', ''))
df1 = df1.withColumn("TWEET", 
                     regexp_replace(col("TWEET"), r'[^\w\s]|quot|amp|\bwww\.\S+|\d+', ''))

# Count tweets with more than 140 characters
df1 = df1.withColumn("tweet_length", length("TWEET"))
print('Total de tweets con más de 140 caracteres:', df1.filter(length("TWEET") > 140).count())
#counts = counts_df.filter(counts_df["tweet_length"] > 140).count()
#print('Total de tweets con más de 140 caracteres:', counts)
df1.show(5)

Total de tweets con más de 140 caracteres: 2


+----+----------+-------------------+-----------+--------------------+------------+
|Indx|        ID|               DATE|DAY_OF_WEEK|               TWEET|tweet_length|
+----+----------+-------------------+-----------+--------------------+------------+
|   0|1467810369|2009-04-06 22:19:45|         01|   awww thats a b...|          74|
|   1|1467810672|2009-04-06 22:19:49|         01|is upset that he ...|         105|
|   2|1467810917|2009-04-06 22:19:53|         01| i dived many tim...|          76|
|   3|1467811184|2009-04-06 22:19:57|         01|my whole body fee...|          47|
|   4|1467811193|2009-04-06 22:19:57|         01| no its not behav...|          87|
+----+----------+-------------------+-----------+--------------------+------------+
only showing top 5 rows



# Write DF in a cvs file

In [14]:
# Store in Hadoop
df1.write.mode("overwrite").option("header", True).csv('/CA2/clean_tweets')

In [12]:
# Store local
df1.coalesce(1).write.option("header", True).csv('file:///home/hduser/Downloads/clean_tweets')
